**Table of contents**<a id='toc0_'></a>    
- 1. [Imports](#toc1_)    
- 2. [Solve DP](#toc2_)    
- 3. [Simulation](#toc3_)    
- 4. [Policy functions](#toc4_)    
- 5. [Save](#toc5_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=false
	minLevel=2
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

## 1. <a id='toc1_'></a>[Imports](#toc0_)

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from NonConvexDurablesModelVFI import NonConvexDurablesModelVFIClass

## 2. <a id='toc2_'></a>[Solve DP](#toc0_)

In [ ]:
model_DP = NonConvexDurablesModelVFIClass()

In [ ]:
model_DP.link_to_cpp()

In [ ]:
model_DP.solve()
model_DP.simulate_R()
model_DP.compute_transfer_func()

In [ ]:
sim = model_DP.sim
par = model_DP.par
vfi = model_DP.vfi

## 3. <a id='toc3_'></a>[Simulation](#toc0_)

In [ ]:
print(f'{sim.R = :12.8f}')

In [ ]:
fig = plt.figure(figsize=(12,8))

ax = fig.add_subplot(2,2,1)
ax.plot(np.mean(model_DP.sim.c, axis=1))
ax.set_title('con')

ax = fig.add_subplot(2,2,2)
ax.plot(np.mean(model_DP.sim.d,axis=1))
ax.set_title('d')

ax = fig.add_subplot(2,2,3)
ax.plot(np.mean(model_DP.sim.DC,axis=1))
ax.set_title('adj')

ax = fig.add_subplot(2,2,4)
ax.plot(np.mean(model_DP.sim.states[...,0],axis=1))
ax.set_title('m')

fig.tight_layout()

## 4. <a id='toc4_'></a>[Policy functions](#toc0_)

In [ ]:
for t in [0,5,10,14]:

    fig = plt.figure(figsize=(6,4))    
    ax = fig.add_subplot(1,1,1)
    
    for i_p in [vfi.Np//5,2*vfi.Np//5]:
        for i_n,ls in zip([vfi.Nn//3,2*vfi.Nn//3],['-','--']):
            y = (1-model_DP.vfi.sol_sav_share_keep[t,i_p,i_n,:]) * model_DP.vfi.m_grid
            ax.plot(vfi.m_grid,y,ls=ls,label=f"p={vfi.p_grid[i_p]:.2f}, n={vfi.n_grid[i_n]:.2f}")
    
    ax.legend()

    ax.set_title(f't={t}')

In [ ]:
for t in [0,5,10,14]:

    fig,ax = plt.subplots(1,2)
    
    # ax = fig.a
    for i_p in [vfi.Np//5,2*vfi.Np//5,3*vfi.Np//5]:
        for i_n,ls in zip([vfi.Nn//3,2*vfi.Nn//3],['-','--']):
            
            x =  vfi.m_grid + (1-par.kappa) * vfi.n_grid[i_n]
            expenditure = (1-vfi.sol_exp_share_adj[t,i_p,i_n]) * x
            c = vfi.sol_c_share_adj[t,i_p,i_n] * expenditure
            d = expenditure - c
            
            ax[0].plot(vfi.m_grid,c,ls=ls,label=f"p={vfi.p_grid[i_p]:.2f}, n={vfi.n_grid[i_n]:.2f}")
            ax[1].plot(vfi.m_grid,d,ls=ls,label=f"p={vfi.p_grid[i_p]:.2f}, n={vfi.n_grid[i_n]:.2f}")

    ax[0].legend()
    ax[0].set_title('c')
    ax[1].set_title('d')

    fig.suptitle(f't={t}')

## 5. <a id='toc5_'></a>[Save](#toc0_)

In [ ]:
#model_DP.save('../output/NonConvexDurablesModel_1D_DP_full.pkl')

In [ ]:
# a. grids for dynamic states

# keeper
vfi.sol_sav_share_keep = None
vfi.sol_v_keep = None

# adjuster
vfi.sol_exp_share_adj = None
vfi.sol_c_share_adj = None
vfi.sol_v_adj = None

vfi.sol_func_evals_keep = None
vfi.sol_flag_keep = None
vfi.sol_func_evals_adj = None
vfi.sol_flag_adj = None

# post-decision
vfi.sol_w = None

In [ ]:
model_DP.save('../output/NonConvexDurablesModel_1D_DP.pkl')